In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from sklearn.decomposition import PCA

data = pd.read_csv("./test_sample.csv")
data.describe()

X = data.iloc[:,1:]
Y = data['Y']

In [2]:
def find_best_model(X, Y, max_vars=491, min_r_squared=0.9):
    for j in range(2, max_vars+1):
        X_j = X.iloc[:, :j]
        X_j = sm.add_constant(X_j)
        model = sm.OLS(Y, X_j).fit()
        r_squared = model.rsquared

        if r_squared > min_r_squared:
            return j

    return None
n_orig = find_best_model(X,Y)
print("smallest number R^2 > 0.9:",n_orig)

smallest number R^2 > 0.9: 320


In [3]:
def pca_analysis(X, n_components=491):
    pca = PCA(n_components=n_components)
    pca.fit(X)
    
    importance = pd.DataFrame({
        'Standard deviation': np.sqrt(pca.explained_variance_),
        'Proportion of Variance': pca.explained_variance_ratio_,
        'Cumulative Proportion': np.cumsum(pca.explained_variance_ratio_)
    }, index=[f"PC{i+1}" for i in range(n_components)])

    scores = pd.DataFrame(
        pca.transform(X),
        columns=[f"PC{i+1}" for i in range(n_components)]
    )   
    return importance, scores

xPCA_importance, factorScores=pca_analysis(X,491)

In [4]:
def rel_imp_me(X, y):
    names = X.columns
    ser = pd.Series(index=names)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()

    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared

    res = pd.DataFrame(columns=['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y, x)[0, 1], axis=0)
    res['first'] = corr ** 2
    sx = X.std()
    res['betasq'] = (lm0.params[names] * sx / np.std(y)) ** 2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res
metrics_PCA = rel_imp_me(factorScores, Y)
print(metrics_PCA)

first_PCA_rank = metrics_PCA["first"].rank(ascending=False, method='first')
print(first_PCA_rank)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14304\2384940573.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser = pd.Series(index=names)


               last         first        betasq         pratt
PC1    5.520450e-03  5.520450e-03  5.531513e-03  5.525979e-03
PC2    1.232491e-02  1.232491e-02  1.234961e-02  1.233725e-02
PC3    4.492875e-04  4.492875e-04  4.501879e-04  4.497375e-04
PC4    3.378590e-02  3.378590e-02  3.385361e-02  3.381974e-02
PC5    3.282607e-03  3.282607e-03  3.289185e-03  3.285895e-03
...             ...           ...           ...           ...
PC487  4.042346e-06  4.042346e-06  4.050447e-06  4.046394e-06
PC488  6.133077e-07  6.133077e-07  6.145367e-07  6.139219e-07
PC489  6.589760e-07  6.589760e-07  6.602966e-07  6.596360e-07
PC490  4.114861e-10  4.114861e-10  4.123107e-10  4.118982e-10
PC491  9.008223e-08  9.008223e-08  9.026275e-08  9.017245e-08

[491 rows x 4 columns]
PC1       41.0
PC2       20.0
PC3      230.0
PC4        2.0
PC5       79.0
         ...  
PC487    438.0
PC488    473.0
PC489    472.0
PC490    491.0
PC491    484.0
Name: first, Length: 491, dtype: float64


In [5]:
metrics_PCA_sort = pd.DataFrame({"Factors" : first_PCA_rank.index,
                                 "Rank" : first_PCA_rank.values}).sort_values(by="Rank") 
orderedFactors = pd.DataFrame(factorScores, columns= metrics_PCA_sort["Factors"])

In [6]:
def rSquar(j, y, X) :
    return sm.OLS(y, sm.add_constant(X.iloc[:,:j])).fit().rsquared

In [ ]:
target_r_squared = 0.9 

n_PCA = None
model_dimensionality_reduction = None

orderedPCA_R2 = [rSquar(j,Y, orderedFactors) for j in range(2,492)]

for j in range(2, 492):
    if orderedPCA_R2[j - 2] >= target_r_squared:
        n_PCA = j
        model_dimensionality_reduction = n_orig - n_PCA
        print("Determination coefficient(R^2):", orderedPCA_R2[j - 2])
        break

print("Smallest number of PCA factors:", n_PCA)
print("Model dimensionality reduction:", model_dimensionality_reduction)
